In [1]:
using JuMP, Ipopt

\title{Lanzamiento de un Cohete}

Un cohete debe ser lanzado de forma vertical y debe, a partir 
del impulso, usado como control, alcanzar la mayor altura 
posible.

Este es un problema de optimización dinámica.

Para este problema tomamos:

- $t$ : tiempo;
- $m(t)$ : masa del cohete;
- $v(t)$ : velocidad del cohete;
- $I(t)$ : impulso del cohete.

Las ecuaciones del movimiento del cohete son
\begin{equation} \label{eq:h_dot}
h' = v,
\end{equation}

\begin{equation} \label{eq:m_dot}
m' = -{v_0}^{-1} I(t),
\end{equation}

\begin{equation} \label{eq:v_dot}
v' = m^{-1} [I(t) - D(v, h)] - G(h),
\end{equation}
donde $h$ es la altura a la que se encuentra el cohete, $v$ es 
su velocidad vertical, $v_0$ es una constante que mide el 
impulso del combustible del cohete, $D(v, h)$ es la resistencia
del aire y $G(h)$ es la fuerza de la gravedad, que se definen 
como siendo
\begin{equation}
G(h) = c_1 h^{-2}
\end{equation}
y
\begin{equation}
D(v, h) = c_2 v^2 e^{-\beta h},
\end{equation}
donde $c_1$ es una constante correspondiente a la fuerza 
gravitacional en la superficie de la tierra, $c_2$ y $\beta$ son
constantes.

In [41]:
mod = Model(with_optimizer(Ipopt.Optimizer))

# Constants
# Note that all parameters in the model have been normalized
# to be dimensionless. See the COPS3 paper for more info.
h_0 = 1    # Initial height
v_0 = 0    # Initial velocity
m_0 = 1    # Initial mass
g_0 = 1    # Gravity at the surface

# Parameters
T_c = 3.5  # Used for thrust
h_c = 500  # Used for drag
v_c = 620  # Used for drag
m_c = 0.6  # Fraction of initial mass left at end

# Derived parameters
c     = 0.5*sqrt(g_0*h_0)  # Thrust-to-fuel mass
m_f   = m_c*m_0            # Final mass
D_c   = 0.5*v_c*m_0/g_0    # Drag scaling
T_max = T_c*g_0*m_0        # Maximum thrust

n = 800   # Time steps
@variable(mod, Δt ≥ 0, start = 1/n)   # Time step
#@expression(t_f, Δt*n)               # Time of flight

# State variables
@variable(mod, v[0:n] ≥ 0)            # Velocity
@variable(mod, h[0:n] ≥ h_0)          # Height
@variable(mod, m_f ≤ m[0:n] ≤ m_0)    # Mass

# Control: thrust
@variable(mod, 0 ≤ T[0:n] ≤ T_max)

# Initial conditions
@NLconstraint(mod, v[0] == v_0)
@NLconstraint(mod, h[0] == h_0)
@NLconstraint(mod, m[0] == m_0)
@NLconstraint(mod, m[n] == m_f)


# @variable(mod, x[i = 1:3])
# @expression(mod, expr[i = 1:3], i * sum(x[j] for j in i:3))

# Forces
# Drag(h,v) = Dc v^2 exp( -hc * (h - h0) / h0 )
@NLexpression(mod, drag[j=0:n], D_c*(v[j]^2)*exp(-h_c*(h[j]-h_0)/h_0))
# Grav(h)   = go * (h0 / h)^2
@NLexpression(mod, grav[j=0:n], g_0*(h_0/h[j])^2)

# Dynamics
for j in 1:n
    # h' = v
    # Rectangular integration
    # @addNLConstraint(mod, h[j] == h[j-1] + Δt*v[j-1])
    # Trapezoidal integration
    @constraint(mod,
        h[j] == h[j-1] + 0.5*Δt*(v[j]+v[j-1]))

    # v' = (T-D(h,v))/m - g(h)
    # Rectangular integration
    # @addNLConstraint(mod, v[j] == v[j-1] + Δt*(
    #                 (T[j-1] - drag[j-1])/m[j-1] - grav[j-1]))
    # Trapezoidal integration
    @NLconstraint(mod,
        v[j] == v[j-1] + 0.5*Δt*(
            (T[j  ] - drag[j  ] - m[j  ]*grav[j  ])/m[j  ] +
            (T[j-1] - drag[j-1] - m[j-1]*grav[j-1])/m[j-1] ))

    # m' = -T/c
    # Rectangular integration
    # @addNLConstraint(mod, m[j] == m[j-1] - Δt*T[j-1]/c)
    # Trapezoidal integration
    @constraint(mod,
        m[j] == m[j-1] - 0.5*Δt*(T[j] + T[j-1])/c)
end

# Provide starting solution
for k in 0:n
    set_start_value(h[k], 1)
    set_start_value(v[k], (k/n)*(1 - (k/n)))
    set_start_value(m[k], (m_f - m_0)*(k/n) + m_0)
    set_start_value(T[k], T_max/2)
end

# Objective: maximize altitude at end of time of flight
@objective(mod, Max, h[n])

# Solve for the control and state
println("Solving...")
optimize!(mod)

# Display results
#println("Solver status: ", status)
println("Max height: ", JuMP.value(h[n]))

Solving...
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    16804
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    39200

Total number of variables............................:     3205
                     variables with only lower bounds:     1603
                variables with lower and upper bounds:     1602
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2404
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_p